In [34]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
sc.stop()
sc = SparkContext()
sqlContext = SQLContext(sc)

rdd = sc.textFile("precipitation-readings-tiny.csv")
parts = rdd.map(lambda l: l.split(";"))
df = parts.map(lambda p: Row(year=p[1].split("-")[0], month=p[1].split("-")[1], prec=float(p[3]),station=int(p[0]) ))
df = sqlContext.createDataFrame(df)
df.registerTempTable("precReadings")

rdd = sc.textFile("stations-Ostergotland.csv")
parts = rdd.map(lambda l: l.split(";"))
df1 = parts.map(lambda p: Row(station=int(p[0]) ))
df1 = sqlContext.createDataFrame(df1)
df1.registerTempTable("stations")

df = df.filter(df.year.between(1933, 2016))
dfavg = df.groupBy("year","month","station").sum("prec").withColumnRenamed("sum(prec)","prec").sort(df.year.desc(), df.month.desc())
df1 = df1.join(dfavg, ["station"], "inner").select(dfavg.prec, dfavg.year, dfavg.month, df1.station)
df1 = df1.groupBy("year","month").avg("prec").withColumnRenamed("avg(temp)","temp").sort(df1.year.desc(), df1.month.desc())

monthAve = df1.rdd
print(monthAve.take(10))
# monthAve.saveAsTextFile("./results2/2q5")

[Row(year='1996', month='10', avg(prec)=8.0)]
